# Simple shear test of Lubby2 Model

In [1]:
import mtest
import numpy as np
import matplotlib.pyplot as plt
import os
plt.rcParams.update({'font.size': 16})
#mtest.setVerboseMode(mtest.VerboseLevel.VERBOSE_QUIET)

In [2]:
A1 = 0.18
Q1 = 54e3
E = 25e3
m1 = 5
nu = 0.27

A2 = 0.
Q2 = 54e3
sref = 1.
Dgrain = 5e-3

In [3]:
build_release = 'mfront --obuild --interface=generic PowerLawLinearCreep.mfront'
os.system(build_release)

Treating target : all
The following library has been built :
- libBehaviour.so :  MohrCoulombAbboSloan_AxisymmetricalGeneralisedPlaneStrain MohrCoulombAbboSloan_AxisymmetricalGeneralisedPlaneStress MohrCoulombAbboSloan_Axisymmetrical MohrCoulombAbboSloan_PlaneStress MohrCoulombAbboSloan_PlaneStrain MohrCoulombAbboSloan_GeneralisedPlaneStrain MohrCoulombAbboSloan_Tridimensional ModCamClay_semiExplParaInit_AxisymmetricalGeneralisedPlaneStrain ModCamClay_semiExplParaInit_AxisymmetricalGeneralisedPlaneStress ModCamClay_semiExplParaInit_Axisymmetrical ModCamClay_semiExplParaInit_PlaneStress ModCamClay_semiExplParaInit_PlaneStrain ModCamClay_semiExplParaInit_GeneralisedPlaneStrain ModCamClay_semiExplParaInit_Tridimensional PowerLawLinearCreep_AxisymmetricalGeneralisedPlaneStrain PowerLawLinearCreep_AxisymmetricalGeneralisedPlaneStress PowerLawLinearCreep_Axisymmetrical PowerLawLinearCreep_PlaneStress PowerLawLinearCreep_PlaneStrain PowerLawLinearCreep_GeneralisedPlaneStrain PowerLawLinearCre

0

In [4]:
def run_temp_sim_BGRa(t_discrete, T0=293.15):
    m    = mtest.MTest()
    m.setMaximumNumberOfSubSteps(1)
    m.setBehaviour('generic', './src/libBehaviour.so', 'PowerLawLinearCreep')
    m.setImposedStress('SXX', 0.0)
    m.setImposedStress('SYY', 0.0)
    m.setImposedStrain('EZZ', -0.01)
    m.setImposedStress('SXZ', 0.0)
    m.setImposedStress('SYZ', 0.0)
    m.setImposedStress('SXY', 0.0)
    m.setMaterialProperty('YoungModulus', E)
    m.setMaterialProperty('PoissonRatio', nu)
    m.setMaterialProperty('PowerLawFactor', A1)
    m.setMaterialProperty('PowerLawEnergy', Q1)
    m.setMaterialProperty('PowerLawExponent', m1)
    m.setMaterialProperty('LinearLawFactor', A2)
    m.setMaterialProperty('LinearLawEnergy', Q2)
    m.setMaterialProperty('ReferenceStress', sref)
    m.setMaterialProperty('SaltGrainSize', Dgrain)
    m.setExternalStateVariable("Temperature", T0)
    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    numerical = np.array([0.])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        numerical = np.append(numerical,s.s1[2]) #Kelvin mapping backwards!
    return numerical

In [5]:
times = np.linspace(0,100,100)

In [ ]:
temps = [20,50,100,150]
fig,ax = plt.subplots(figsize=(12,8))
for Temp in temps:
    ax.plot(times[1:],run_temp_sim_BGRa(times,273.15+Temp)[1:],label='$T$ = %i °C' %Temp)
ax.set_xlabel('$t$ / d')
ax.set_ylabel('$\\sigma_{zz}$ / MPa')
fig.tight_layout();
ax.legend();

In [ ]:
def run_grain_sim_PLL(t_discrete, D=5e-3):
    Dgrain = D
    m    = mtest.MTest()
    m.setMaximumNumberOfSubSteps(1)
    m.setBehaviour('generic', './src/libBehaviour.so', 'PowerLawLinearCreep')
    m.setImposedStress('SXX', 0.0)
    m.setImposedStress('SYY', 0.0)
    m.setImposedStrain('EZZ', -0.01)
    m.setImposedStress('SXZ', 0.0)
    m.setImposedStress('SYZ', 0.0)
    m.setImposedStress('SXY', 0.0)
    m.setMaterialProperty('YoungModulus', E)
    m.setMaterialProperty('PoissonRatio', nu)
    m.setMaterialProperty('PowerLawFactor', A1*0)
    m.setMaterialProperty('PowerLawEnergy', Q1)
    m.setMaterialProperty('PowerLawExponent', m1)
    m.setMaterialProperty('LinearLawFactor', A1)
    m.setMaterialProperty('LinearLawEnergy', Q1)
    m.setMaterialProperty('ReferenceStress', sref)
    m.setMaterialProperty('SaltGrainSize', Dgrain)
    m.setExternalStateVariable("Temperature", 393.15)
    s = mtest.MTestCurrentState()
    wk = mtest.MTestWorkSpace()
    m.completeInitialisation()
    m.initializeCurrentState(s)
    m.initializeWorkSpace(wk)
    numerical = np.array([0.])
    for i in range(len(t_discrete)-1):
        m.execute(s, wk, t_discrete[i], t_discrete[i + 1])
        numerical = np.append(numerical,s.s1[2]) #Kelvin mapping backwards!
    return numerical

In [ ]:
Ds = [5e-3,5e-2,5e-1]
fig,ax = plt.subplots(figsize=(12,8))
for d in Ds:
    ax.plot(times[1:],run_grain_sim_PLL(times,d)[1:],label='$D$ = %.3f mm' %(d*1e3))
ax.set_xlabel('$t$ / d')
ax.set_ylabel('$\\sigma_{zz}$ / MPa')
fig.tight_layout();
ax.legend();